## Install Dependencies



In [1]:
!pip install -r requirements.txt

  Using cached Pint-0.18-py2.py3-none-any.whl (209 kB)
  Using cached eep153_tools-0.11-py2.py3-none-any.whl (4.4 kB)
Processing /home/jovyan/.cache/pip/wheels/20/7e/30/7d702acd6a1e89911301cd9dbf9cb9870ca80c0e64bc2cde23/gnupg-2.3.1-py3-none-any.whl
  Attempting uninstall: pint
    Found existing installation: Pint 0.17
    Uninstalling Pint-0.17:
      Successfully uninstalled Pint-0.17


In [2]:
import fooddatacentral as fdc

In [3]:
from  scipy.optimize import linprog as lp
import numpy as np
import warnings
import pandas as pd
from eep153_tools.sheets import read_sheets

## Utilities + Load and Clean Data

In [4]:
apikey = 'MfcTfizjo11bsqJeJCn9Tb7RdKPQxJRjJSvTKElr' # This is Kaitlin's API key!

In [5]:
food_name = "orange juice"

fdc.search(apikey,food_name)

,fdcId,description,lowercaseDescription,dataType,gtinUpc,publishedDate,brandOwner,ingredients,marketCountry,foodCategory,...,foodAttributeTypes,foodVersionIds,brandName,packageWeight,subbrandName,commonNames,additionalDescriptions,ndbNumber,foodCode,foodCategoryId
0,579417,ORANGE JUICE,orange juice,Branded,048500008607,2019-04-01,"Tropicana Products, Inc.",ORANGE JUICE FROM CONCENTRATE (FILTERED WATER ...,United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2051634,ORANGE JUICE,orange juice,Branded,846656018774,2021-10-28,Agricultural Marketing Company Limited,"WATER, ORANGE JUICE CONCENTRATE, SUGAR, PECTIN...",United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],FORTUNATA,33.81 fl oz/1 L,NaN,NaN,NaN,NaN,NaN,NaN
2,2091685,ORANGE JUICE,orange juice,Branded,085239040430,2021-10-28,Target Stores,"WATER, ORANGE JUICE CONCENTRATE, TRICALCIUM PH...",United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],MARKET PANTRY,96 fl oz/3 Quart/2.8 L,NaN,NaN,NaN,NaN,NaN,NaN
3,2098640,ORANGE JUICE,orange juice,Branded,072730292413,2021-10-28,"Prairie Farms Dairy, Inc.","ORANGE JUICE, CALCIUM PHOSPHATE* AND VITAMIN D3.*",United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],PRAIRIE FARMS,1.75 L/1.85 Quart/59 fl o,NaN,NaN,NaN,NaN,NaN,NaN
4,2104399,ORANGE JUICE,orange juice,Branded,099482407018,2021-10-28,"Whole Foods Market, Inc.",ORANGE JUICE.,United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],365 EVERYDAY VALUE,64 fl oz/1.89 L,NaN,NaN,NaN,NaN,NaN,NaN
5,2104927,ORANGE JUICE,orange juice,Branded,607880101515,2021-10-28,"Bruno's Supermarkets, Inc.","WATER, ORANGE JUICE CONCENTRATE, TRICALCIUM PH...",United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],SE GROCERS,0.5 GAL/1.89 L,NaN,NaN,NaN,NaN,NaN,NaN
6,2073024,ORANGE JUICE,orange juice,Branded,011161460183,2021-10-28,"Topco Associates, Inc.","FILTERED WATER, ORANGE JUICE CONCENTRATE.",United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],SHURFINE,32 fl oz/1 Quart/946 mL,NaN,NaN,NaN,NaN,NaN,NaN
7,2073232,ORANGE JUICE,orange juice,Branded,030034080808,2021-10-28,"Giant Eagle, Inc.","100% PURE PASTEURIZED ORANGE JUICE, TRICALCIUM...",United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],GIANT EAGLE,89 fl oz/2.78 Quart/2.63,NaN,NaN,NaN,NaN,NaN,NaN
8,2133897,ORANGE JUICE,orange juice,Branded,030034918477,2021-10-28,"Giant Eagle, Inc.",ORANGE JUICE.,United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],GIANT EAGLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,579415,ORANGE JUICE,orange juice,Branded,048500009758,2019-04-01,"Tropicana Products, Inc.",ORANGE JUICE FROM CONCENTRATE (FILTERED WATER ...,United States,"Fruit & Vegetable Juice, Nectars & Fruit Drinks",...,[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
id =  455481   # Put an FDC ID HERE!

fdc.nutrients(apikey,fdc_id=id)

,Quantity,Units
"Iron, Fe",2.25,mg
"Calcium, Ca",0.00,mg
"Vitamin C, total ascorbic acid",0.00,mg
Cholesterol,0.00,mg
"Vitamin A, IU",0.00,IU
Total lipid (fat),50.00,g
"Fatty acids, total trans",0.00,g
"Sugars, total including NLEA",9.38,g
"Carbohydrate, by difference",21.88,g
"Fiber, total dietary",6.20,g


In [34]:
SHEETs = [# options: Berkeley, Whole; Berkeley, Prepared; Atlanta, Whole; Atlanta, Prepared 
          ("https://docs.google.com/spreadsheets/d/1j38SlusoWvBtohgU-PAondBKihr1tCpjfLcgit7Ic_s/edit#gid=0","Atlanta, Whole"), 
         ]

In [35]:
df = read_sheets(SHEETs[0][0])[SHEETs[0][1]]
df.drop(['Name of person', 'Link'], axis=1)

Key available for students@eep153.iam.gserviceaccount.com.


,Food,Quantity,Units,Price,Date,Location,FDC
0,egg,25.50,oz,2.59,[2022-2-23],Atlanta 30303,748967
1,banana,2.00,lbs,1.29,[2022-2-23],Atlanta 30303,1102653
2,cheddar cheese,8.00,oz,2.19,[2022-2-23],Atlanta 30303,1098007
3,chicken breast,1.50,lbs,8.99,[2022-2-23],Atlanta 30303,1098461
4,spaghetti noodles,16.00,oz,1.44,[2022-2-23],Atlanta 30303,1101523
5,broccoli,12.00,oz,1.59,[2022-2-23],Atlanta 30303,1103170
6,carrots,5.00,lbs,4.19,[2022-2-23],Atlanta 30303,1103193
7,cucumber,16.00,oz,3.09,[2022-2-23],Atlanta 30303,1103352
8,peanut butter,40.00,oz,4.63,[2022-2-23],Atlanta 30303,1100559
9,whole milk,128.00,oz,3.19,[2022-2-23],Atlanta 30303,1097512


In [36]:
D = {}
count = 0
for food in df.Food.tolist():
    try:
        FDC = df.loc[df.Food==food,:].FDC[count]
        count+=1
        D[food] = fdc.nutrients(apikey,FDC).Quantity
    except AttributeError: 
        warnings.warn("Couldn't find FDC Code %s for food %s." % (food,FDC))        

FoodNutrients = pd.DataFrame(D,dtype=float)

In [37]:
FoodNutrients[FoodNutrients.index == 'Cholesterol']

,egg,banana,cheddar cheese,chicken breast,spaghetti noodles,broccoli,carrots,cucumber,peanut butter,whole milk,...,onions,black beans,corn,tomatoes,apples,oranges,strawberries,oats,spinach,tuna
Cholesterol,411.0,0.0,100.0,96.0,29.0,0.0,0.0,0.0,0.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0


In [38]:
# Convert food quantities to FDC units
df['FDC Quantity'] = df[['Quantity','Units']].T.apply(lambda x : fdc.units(x['Quantity'],x['Units']))

# Now may want to filter df by time or place--need to get a unique set of food names.
df['FDC Price'] = df['Price']/df['FDC Quantity']

df.dropna(how='any') # Drop food with any missing data

# To use minimum price observed
Prices = df.groupby('Food',sort=False)['FDC Price'].min()

/opt/conda/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1990: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


# Deliverable: Dietary Reference Intakes

In [12]:
def DRI(age, sex):
    '''
    Description: Function that retrieves Dietary Reference Intakes or Recommended Daily Allowances for a
        population of interest described by the input characteristics

    Inputs:
        Age: The age of interest, in years (integer)
        Sex: The sex of interest, 'M' or 'F' (string)
        
    Output: Returns a pandas.Series of DRI or RDA of a variety of nutrients for our population of interest
    '''
    
    # Make a combined dataframe with min and max nutrients
    nut_min = pd.read_csv('./diet_minimums.csv').set_index('Nutrition').iloc[:, 2:]
    nut_max = pd.read_csv('./diet_maximums.csv').set_index('Nutrition').iloc[:, 2:]
    nut = pd.concat([nut_min, -nut_max])
    
    # Case 1: Child
    if age in range(1, 4):
        return nut.loc[:,'C 1-3']
    
    # Case 2: Not a child
    # Determine the age-range; Filter data by desired age-range and sex
    age_ranges = [[1,4], [4,9], [9,14], [14,19], [19,31], [31,51], [51,1000]]
    for r in age_ranges:
        if age in range(r[0], r[1]):
            return nut.filter(regex = sex).filter(regex = ' ' + str(r[0])).iloc[:, 0]
        
def grouping(age, sex):
    # Case 1: Child
    if age in range(1, 4):
        return 'C 1-3'
    
    # Case 2: Not a child
    # Determine the age-range; Filter data by desired age-range and sex
    age_ranges = [[1,4], [4,9], [9,14], [14,19], [19,31], [31,51], [51,1000]]
    for r in age_ranges:
        if age in range(r[0], r[1]):
            return sex + ' ' + str(r[0]) + '-' + str(r[1]-1)

# Deliverable: Subsistence Solver

In [13]:
def solve_subsistence_problem(FoodNutrients,Prices,diet_min,diet_max,max_weight=23,tol=1e-6):
    """Solve Stigler's Subsistence Cost Problem.

    Inputs:
       - FoodNutrients : A pd.DataFrame with rows corresponding to foods, columns to nutrients.
       - Prices : A pd.Series of prices for different foods
       - diet_min : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing minimum intakes.
       - diet_max : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing maximum intakes.
       - max_weight : Maximum weight (in hectograms) allowed for diet.
       - tol : Solution values smaller than this in absolute value treated as zeros.
       
    """
    p = Prices.apply(lambda x:x.magnitude).dropna()

    # Compile list that we have both prices and nutritional info for; drop if either missing
    use = p.index.intersection(FoodNutrients.columns)
    p = p[use]

    # Drop nutritional information for foods we don't know the price of,
    # and replace missing nutrients with zeros.
    Aall = FoodNutrients[p.index].fillna(0)

    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[Aall.index.intersection(diet_min.index)]

    Amax = Aall.loc[Aall.index.intersection(diet_max.index)]

    # Minimum requirements involve multiplying constraint by -1 to make <=.
    A = pd.concat([Amin,
                   -Amax])

    b = pd.concat([diet_min,
                   -diet_max]) # Note sign change for max constraints

    # Make sure order of p, A, b are consistent
    A = A.reindex(p.index,axis=1)
    A = A.reindex(b.index,axis=0)

    if max_weight is not None:
        # Add up weights of foods consumed
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight
        
    # Now solve problem!  (Note that the linear program solver we'll use assumes
    # "less-than-or-equal" constraints.  We can switch back and forth by
    # multiplying $A$ and $b$ by $-1$.)

    result = lp(p, -A, -b, method='interior-point')

    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        warnings.warn(result.message)
        result.diet = pd.Series(result.x,index=p.index)*np.nan  

    return result

## Dietary Requirements



In [39]:
from eep153_tools.sheets import read_sheets

# using Dietary Requirements sheet from class
DRI_url = "https://docs.google.com/spreadsheets/d/1XJRHTnxNJwrUXperIhwrwDp1HcVxPEVoQobYDmjg9Qw/"

DRIs = read_sheets(DRI_url)

# Define *minimums*
diet_min = DRIs['diet_minimums'].set_index('Nutrition')

# Define *maximums*
diet_max = DRIs['diet_maximums'].set_index('Nutrition')

# diet max but with forgiving values - basically forces the output to tell us the end values for sugar, chol, salt
diet_max_forgiving = DRIs['diet_maximums_forgiving'].set_index('Nutrition')

Key available for students@eep153.iam.gserviceaccount.com.


In [40]:
# diet_max = diet_max[1:2]

In [41]:
diet_max

,Source,C 1-3,F 4-8,M 4-8,F 9-13,M 9-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
Nutrition,,,,,,,,,,,,,,
"Sodium, Na",UL,1500,1900,1900,2200,2200,2300,2300,2300,2300,2300,2300,2300,2300
"Sugars, total including NLEA",online,36,36,36,36,36,36,36,36,36,36,36,36,36
Cholesterol,online,300,300,300,300,300,300,300,300,300,300,300,300,300


## Solving for the minimum diet


In [42]:
group = 'M 19-30'
tol = 1e-6

result = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol)

print("Cost of diet for %s is $%4.2f per day.\n" % (group,result.fun))

# Put back into nice series
diet = result.diet

print("\nDiet (in 100s of grams or milliliters):")
print(diet[diet >= tol])  # Drop items with quantities less than precision of calculation.
print()

tab = pd.DataFrame({"Outcome":np.abs(result.A).dot(diet),"Recommendation":np.abs(result.b)})
print("\nWith the following nutritional outcomes of interest:")
print(tab)
print()

print("\nConstraining nutrients are:")
excess = tab.diff(axis=1).iloc[:,1]
print(excess.loc[np.abs(excess) < tol*100].index.tolist())

Cost of diet for M 19-30 is $2.98 per day.


Diet (in 100s of grams or milliliters):
egg              0.602262
carrots          0.646759
peanut butter    0.422837
whole milk       3.230511
almond milk      1.466958
cabbage          1.043377
potatoes         4.567995
black beans      2.864015
oats             1.301787
dtype: float64


With the following nutritional outcomes of interest:
                                    Outcome  Recommendation
Nutrition                                                  
Energy                          2400.000025          2400.0
Protein                           81.393140            56.0
Fiber, total dietary              54.918780            33.6
Folate, DFE                      419.728078           400.0
Calcium, Ca                     1000.000022          1000.0
Carbohydrate, by difference      289.092622           130.0
Iron, Fe                          17.923288             8.0
Magnesium, Mg                    582.669615           400.0
Niacin     

## Effects of Price Changes on Subsistence Diet Cost



As prices change, we should expect the minimum cost diet to also
  change.  The code below creates a graph which changes prices away
  from the \`base&rsquo; case one food at a time, and plots changes in total
  diet cost.



In [43]:
import cufflinks as cf
cf.go_offline()

scale = [.5,.6,.7,.8,.9,1.,1.1,1.2,1.3,1.4,1.5]

cost0 = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol).fun

Price_response={}
for s in scale:
    cost = {}
    for i,p in enumerate(Prices):
        my_p = Prices.copy()
        my_p[i] = p*s
        result = solve_subsistence_problem(FoodNutrients,my_p,diet_min[group],diet_max[group],tol=tol)
        cost[Prices.index[i]] = np.log(result.fun/cost0)
    Price_response[np.log(s)] = cost

Price_response = pd.DataFrame(Price_response).T
Price_response.iplot(xTitle='change in log price',yTitle='change in log cost')

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning:

The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.



## Effects of Price Changes on Subsistence Diet Composition



The code below creates a graph which changes prices just for *one* food,
  and traces out the effects of this change on all the foods consumed.



In [33]:
import cufflinks as cf
cf.go_offline()

ReferenceGood = 'Banana'

scale = [0.5,0.75,0.9,1.,1.1,1.2,1.3,1.4,1.5,2,4]

cost0 = solve_subsistence_problem(FoodNutrients,Prices,diet_min[group],diet_max[group],tol=tol).fun

my_p = Prices.copy()

diet = {}
for s in scale:

    my_p[ReferenceGood] = Prices[ReferenceGood]*s
    result = solve_subsistence_problem(FoodNutrients,my_p,diet_min[group],diet_max[group],tol=tol)
    diet[my_p[ReferenceGood]] = result.diet

Diet_response = pd.DataFrame(diet).T
Diet_response.index.name = '%s Price' % ReferenceGood

Diet_response.reset_index(inplace=True)

# Get rid of units for index (cufflinks chokes)
Diet_response['%s Price' % ReferenceGood] = Diet_response['%s Price' % ReferenceGood].apply(lambda x: x.magnitude)

Diet_response = Diet_response.set_index('%s Price' % ReferenceGood)

# Just look at goods consumed in quantities greater than error tolerance
Diet_response.loc[:,(Diet_response>tol).sum()>0].iplot(xTitle='%s Price' % ReferenceGood,yTitle='Hectograms')

## Effects of Price Changes on Subsistence Diet Nutrition



The code below creates a graph which uses the food price changes
  described above, but maps into nutrients.



In [ ]:
# Matrix product maps quantities of food into quantities of nutrients
NutrientResponse = Diet_response@FoodNutrients.T

# Drop columns of missing nutrients
NutrientResponse = NutrientResponse.loc[:,NutrientResponse.count()>0]
NutrientResponse.iplot(xTitle='%s Price' % ReferenceGood,yTitle='Hectograms')

## Adding Constraint on Total Weight



At least at some prices the minimum cost subistence diet involves
eating unreasonable amounts of food (e.g., 10 kilograms of cabbage per
day).  We can easily add an additional constraint of the form
$$
     \sum x_i \leq \text{max weight}
$$
to our linear programming problem since it&rsquo;s just another linear
inequality, and this may give us more realistic results.



### Price Changes and Subsistence Diet Composition with Weight Constraint



Re-do our analysis of changing prices, but with a constraint that
  total weight of diet must be less that 12 hectograms (1.2 kg).



In [1]:
import cufflinks as cf
cf.go_offline()

ReferenceGood = 'Spinach'

scale = [0.5,0.75,0.9,1.,1.1,1.2,1.3,1.4,1.5,2,4]

cost0 = solve_subsistence_problem(FoodNutrients,Prices,
                                  diet_min[group],diet_max[group],max_weight=10,tol=tol).fun

my_p = Prices.copy()

diet = {}
for s in scale:

    my_p[ReferenceGood] = Prices[ReferenceGood]*s
    result = solve_subsistence_problem(FoodNutrients,my_p,
                                       diet_min[group],diet_max[group],max_weight=10,tol=tol)
    diet[my_p[ReferenceGood]] = result.diet

Diet_response = pd.DataFrame(diet).T
Diet_response.index.name = '%s Price' % ReferenceGood

Diet_response.reset_index(inplace=True)

# Get rid of units for index (cufflinks chokes)
Diet_response['%s Price' % ReferenceGood] = Diet_response['%s Price' % ReferenceGood].apply(lambda x: x.magnitude)

Diet_response = Diet_response.set_index('%s Price' % ReferenceGood)

# Just look at goods consumed in quantities greater than error tolerance
Diet_response.loc[:,(Diet_response>tol).sum()>0].iplot(xTitle='%s Price' % ReferenceGood,yTitle='Hectograms')